In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from keras.models import Sequential
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from keras.layers import Dense, Activation,Layer,Lambda
import io
seed=8
np.random.seed(seed) # For Reproducibility

# Reading data
uploaded = files.upload()
forestfire = pd.read_csv(io.StringIO(uploaded['forestfires.csv'].decode('utf-8')))
forestfire.head(5)

Saving forestfires.csv to forestfires (6).csv


,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small


In [99]:
forestfire["month"]=preprocessing.LabelEncoder().fit_transform(forestfire["month"])
forestfire["day"]=preprocessing.LabelEncoder().fit_transform(forestfire["day"])
forestfire["size_category"]=preprocessing.LabelEncoder().fit_transform(forestfire["size_category"])
columns=['month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind','rain','size_category', 'area']
modified_forestfire=forestfire[columns]
features=columns[0:11]
target=columns[11]
print(modified_forestfire.isnull().sum())  #No NA Values present in the Dataset
modified_forestfire.head(5)

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
size_category    0
area             0
dtype: int64


,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,size_category,area
0,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,1,0.0
1,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,1,0.0
2,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,1,0.0
3,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,1,0.0
4,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,1,0.0


In [0]:
Train,Test = train_test_split(modified_forestfire,test_size = 0.15,random_state=42)

def model_building(hidden_layer):
    
    model = Sequential()
    
    for i in range(1,len(hidden_layer)-1):
        if (i==1):
            model.add(Dense(hidden_layer[i],input_dim=hidden_layer[0],kernel_initializer="uniform",activation="relu"))
        else:
            model.add(Dense(hidden_layer[i],activation="relu"))
    model.add(Dense(hidden_layer[-1]))
    model.compile(loss="mean_squared_error",optimizer="adam",metrics = ["accuracy"])
    return model

In [107]:
seed=7
np.random.seed(seed)
neuralnetwork_model = model_building([11,40,30,20,1])
neuralnetwork_model.fit(np.array(Train[features]),np.array(Train[target]),epochs=700,batch_size=5)
train_predictions = neuralnetwork_model.predict(np.array(Train[features]))
train_predictions = pd.Series([i[0] for i in train_predictions])
test_predictions = neuralnetwork_model.predict(np.array(Test[features]))
test_predictions = pd.Series([i[0] for i in test_predictions])
train_rmse = np.sqrt(np.mean((train_predictions-Train[target])**2))
test_rmse = np.sqrt(np.mean((test_predictions-Test[target])**2))

Epoch 1/700
439/439 [==============================] - 0s 448us/step - loss: 2006.6967 - accuracy: 0.0046
Epoch 2/700
439/439 [==============================] - 0s 220us/step - loss: 1985.6666 - accuracy: 0.0000e+00
Epoch 3/700
439/439 [==============================] - 0s 219us/step - loss: 1982.0473 - accuracy: 0.0000e+00
Epoch 4/700
439/439 [==============================] - 0s 225us/step - loss: 1975.4930 - accuracy: 0.0000e+00
Epoch 5/700
439/439 [==============================] - 0s 225us/step - loss: 1979.4744 - accuracy: 0.0000e+00
Epoch 6/700
439/439 [==============================] - 0s 244us/step - loss: 1979.0843 - accuracy: 0.0000e+00
Epoch 7/700
439/439 [==============================] - 0s 281us/step - loss: 1982.3545 - accuracy: 0.0023
Epoch 8/700
439/439 [==============================] - 0s 237us/step - loss: 1960.5239 - accuracy: 0.0023
Epoch 9/700
439/439 [==============================] - 0s 223us/step - loss: 1971.3602 - accuracy: 0.0000e+00
Epoch 10/700
439/439 [

In [108]:
print("RMSE Value of Neural-Network Model: ",test_rmse) #Low-RMSE Value
print("Correlation Co-efficient Value between Predictions and Actual Values: \n ",np.corrcoef(train_predictions,Train[target])) #High Correlation Co-efficient shows that the predictions and
# Actual Values are almost Similar to each other which shows the model performs good


RMSE Value of Neural-Network Model:  4.001474311444047
Correlation Co-efficient Value between Predictions and Actual Values: 
  [[1.         0.92134809]
 [0.92134809 1.        ]]
